In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
driver = webdriver.Chrome('C:/Users/bandi/Desktop/Text Analytics/TA Session/chromedriver_win32/chromedriver')
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702')

In [3]:
#Creating empty data frame to store user_id, dates and comments from ~5K users.
comments = pd.DataFrame(columns = ['Date','user_id','comments'])

j = 702
while (j>=1):
    # Running while loop only till we get 5K comments 
    if (len(comments)<5000):
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p' + str(j)
        driver.get(url)
        ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
        comment_ids = []
        for i in ids:
            comment_ids.append(i.get_attribute('id'))

        for x in comment_ids:
            #Extract dates from for each user on a page
            user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
            date = user_date.get_attribute('title')

            #Extract user ids from each user on a page
            user_title = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[1]')[0]
            userid = user_title.get_attribute('title') 

            #Extract Message for each user on a page
            user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]')[0]
            comment = user_message.text
           #Adding date, userid and comment for each user in a dataframe    
            comments.loc[len(comments)] = [date,userid,comment]
        j=j-1
    else:
        break
        

In [177]:
import copy
comments_copy = copy.deepcopy(comments)

,Date,user_id,comments
0,"September 7, 2018 4:01PM",dino001,If they keep it around in next four-five years...
1,"September 7, 2018 9:43PM",circlew,The lease rate is the factor that stops me col...
2,"September 8, 2018 12:25PM",qbrozen,circlew said:\nThe lease rate is the factor th...
3,"September 9, 2018 3:51AM",FlightNurse2,roadburner said:\nshow previous quotes\n\n\nTh...
4,"September 9, 2018 1:01PM",roadburner,"Again, the local dealer are hopeless at best(W..."
5,"September 9, 2018 3:19PM",FlightNurse2,"roadburner said:\nAgain, the local dealer are ..."
6,"September 9, 2018 4:49PM",roadburner,"Thanks, but I am more than aware of the distin..."
7,"September 9, 2018 5:38PM",dino001,"roadburner said:\nThanks, but I am more than a..."
8,"August 7, 2018 11:51PM",benjaminh,One of the features I like on my TLX's stereo ...
9,"August 8, 2018 12:00AM",andres3,benjaminh said:\nOne of the features I like on...


In [14]:
def remove_space(s):
    return s.replace("\n","")
comments_copy['comments'] = comments_copy['comments'].apply(remove_space)

In [22]:
comments_copy.to_csv('comments.csv', header=True, sep=',') 